In [4]:
%matplotlib inline

import pandas as pd
import seaborn as sns
import numpy as np

from matplotlib import pyplot as plt

from sklearn.ensemble import RandomForestClassifier, BaggingClassifier, GradientBoostingClassifier, BaggingClassifier, VotingClassifier
from sklearn.svm import SVC, LinearSVC, NuSVC
from sklearn.preprocessing import label_binarize, normalize, LabelEncoder
from sklearn.metrics import log_loss, classification_report
from sklearn.decomposition import PCA

from sklearn.cross_validation import cross_val_score, KFold, StratifiedKFold, train_test_split
from sklearn.feature_selection import RFECV
from sklearn.grid_search import GridSearchCV
from sklearn.linear_model import SGDClassifier
from sklearn.calibration import CalibratedClassifierCV


In [5]:
df = pd.read_csv('../input/train.csv')
df.head()

,AnimalID,Name,DateTime,OutcomeType,OutcomeSubtype,AnimalType,SexuponOutcome,AgeuponOutcome,Breed,Color
0,A671945,Hambone,2014-02-12 18:22:00,Return_to_owner,NaN,Dog,Neutered Male,1 year,Shetland Sheepdog Mix,Brown/White
1,A656520,Emily,2013-10-13 12:44:00,Euthanasia,Suffering,Cat,Spayed Female,1 year,Domestic Shorthair Mix,Cream Tabby
2,A686464,Pearce,2015-01-31 12:28:00,Adoption,Foster,Dog,Neutered Male,2 years,Pit Bull Mix,Blue/White
3,A683430,NaN,2014-07-11 19:09:00,Transfer,Partner,Cat,Intact Male,3 weeks,Domestic Shorthair Mix,Blue Cream
4,A667013,NaN,2013-11-15 12:52:00,Transfer,Partner,Dog,Neutered Male,2 years,Lhasa Apso/Miniature Poodle,Tan


In [6]:
df['has_name'] = pd.notnull(df['Name']).astype('bool8')
#####
df.DateTime = pd.to_datetime(df.DateTime)
df['year'] = df.DateTime.dt.year.astype('category')
df['month'] = df.DateTime.dt.month.astype('category')
df['weekday'] = df.DateTime.dt.weekday.astype('category')
df['week_end'] = df.weekday.isin([5,6])
df['hour'] = df.DateTime.dt.hour.astype('category')
def hour_period_map(x):
    if x in [7,8,9,10,11,12]:
        return 'morning'
    elif x in [13,14,15,16,17,18]:
        return 'afternoon'
    elif x in [19,20,21,22,23]:
        return 'night'
    else:  # [0,1,2,3,4,5,6]
        return 'dawn'
    
df['period'] = df.hour.apply(hour_period_map)
#####
df['is_dog'] = (df['AnimalType'] == 'Dog')
####
foo = df['SexuponOutcome'].str.split(expand=True)
df['condition'] = foo[0].map({'Neutered': 'operated', 'Spayed':'operated', 'Intact': 'intact', 'Unknown': 'unknown'})
df['is_male'] = (foo[1] == 'Male')
#####
time_multiplier = dict(year = 365, years=365, weeks = 7, month = 30, months=30, days = 1, week = 7, day= 1)
foo = df['AgeuponOutcome'].str.split(expand=True)
age = foo[0].astype('float32')
period = foo[1]
period = period.map(time_multiplier)
df['age_in_days'] = age * period
df['age_in_years'] = (df['age_in_days'] / 365).round()
df['age_in_months'] = (df['age_in_days'] / 30).round()
#####
df['pure_breed'] = df.Breed.apply(lambda x : 'mix' not in x.lower()).astype('bool8')
####
cores = df.Color.str.split('/', expand=True)

df['cores_1'] = cores[0].str.split(expand=True).ix[:,0]
df['cores_2'] = cores[1].str.split(expand=True).ix[:,0]
df['bi_color'] =  df.Color.str.contains('/').astype('bool8')
####
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26729 entries, 0 to 26728
Data columns (total 27 columns):
AnimalID          26729 non-null object
Name              19038 non-null object
DateTime          26729 non-null datetime64[ns]
OutcomeType       26729 non-null object
OutcomeSubtype    13117 non-null object
AnimalType        26729 non-null object
SexuponOutcome    26728 non-null object
AgeuponOutcome    26711 non-null object
Breed             26729 non-null object
Color             26729 non-null object
has_name          26729 non-null bool
year              26729 non-null category
month             26729 non-null category
weekday           26729 non-null category
week_end          26729 non-null bool
hour              26729 non-null category
period            26729 non-null object
is_dog            26729 non-null bool
condition         26728 non-null object
is_male           26729 non-null bool
age_in_days       26711 non-null float64
age_in_years      26711 non-null float64
a

In [20]:
df2 = df[['DateTime',
          'has_name', 
          'year',
          'month', 
          'weekday', 
          #'weekend',
          #'period', 
          'hour',
          'is_dog', 
          'condition', 
          'is_male', 
          'age_in_days', 
          'pure_breed', 
          'cores_1', 
          'cores_2', 
          'bi_color', 
          'OutcomeType']]
df2.unstack()
df2
#df2.set_index('DateTime',inplace=True)
#df2

,DateTime,has_name,year,month,weekday,hour,is_dog,condition,is_male,age_in_days,pure_breed,cores_1,cores_2,bi_color,OutcomeType
0,2014-02-12 18:22:00,True,2014,2,2,18,True,operated,True,365.0,False,Brown,White,True,Return_to_owner
1,2013-10-13 12:44:00,True,2013,10,6,12,False,operated,False,365.0,False,Cream,None,False,Euthanasia
2,2015-01-31 12:28:00,True,2015,1,5,12,True,operated,True,730.0,False,Blue,White,True,Adoption
3,2014-07-11 19:09:00,False,2014,7,4,19,False,intact,True,21.0,False,Blue,None,False,Transfer
4,2013-11-15 12:52:00,False,2013,11,4,12,True,operated,True,730.0,True,Tan,None,False,Transfer
5,2014-04-25 13:04:00,True,2014,4,4,13,True,intact,False,30.0,True,Black,Tan,True,Transfer
6,2015-03-28 13:11:00,True,2015,3,5,13,False,intact,True,21.0,False,Blue,None,False,Transfer
7,2015-04-30 17:02:00,False,2015,4,3,17,False,unknown,False,21.0,False,Brown,None,False,Transfer
8,2014-02-04 17:17:00,True,2014,2,1,17,True,operated,False,150.0,False,Red,White,True,Adoption
9,2014-05-03 07:48:00,False,2014,5,5,7,True,operated,False,365.0,True,White,None,False,Adoption


,has_name,year,month,weekday,hour,is_dog,condition,is_male,age_in_days,pure_breed,cores_1,cores_2,bi_color,OutcomeType
DateTime,,,,,,,,,,,,,,
2014-02-12 18:22:00,True,2014,2,2,18,True,operated,True,365.0,False,Brown,White,True,Return_to_owner
2013-10-13 12:44:00,True,2013,10,6,12,False,operated,False,365.0,False,Cream,None,False,Euthanasia
2015-01-31 12:28:00,True,2015,1,5,12,True,operated,True,730.0,False,Blue,White,True,Adoption
2014-07-11 19:09:00,False,2014,7,4,19,False,intact,True,21.0,False,Blue,None,False,Transfer
2013-11-15 12:52:00,False,2013,11,4,12,True,operated,True,730.0,True,Tan,None,False,Transfer
2014-04-25 13:04:00,True,2014,4,4,13,True,intact,False,30.0,True,Black,Tan,True,Transfer
2015-03-28 13:11:00,True,2015,3,5,13,False,intact,True,21.0,False,Blue,None,False,Transfer
2015-04-30 17:02:00,False,2015,4,3,17,False,unknown,False,21.0,False,Brown,None,False,Transfer
2014-02-04 17:17:00,True,2014,2,1,17,True,operated,False,150.0,False,Red,White,True,Adoption


In [4]:

df2 = pd.get_dummies(df2, columns=['year', 
                                   'month', 
                                   'weekday', 
                                   #'weekend',
                                   #'period', 
                                   'hour',                                   
                                   'condition', 
                                   'cores_1', 
                                   'cores_2'])
df2.dropna(axis=0, inplace=True)
y = df2['OutcomeType']
x = df2.drop('OutcomeType', axis=1)
#x_norm = normalize(x, 'l2')

In [ ]:
rfc_params = {
    'n_estimators' : np.linspace(100, 300, 3, dtype='int'),
    'max_depth': np.linspace(20, 50, 5, dtype='int'),
    'min_samples_leaf': np.linspace(1, 3, 2, dtype='int'),   
    'class_weight' : [None, 'balanced'],
}
rf_grid = GridSearchCV(RandomForestClassifier(), rfc_params, verbose=3, scoring='log_loss', n_jobs=4)
rf_grid.fit(x, y)
print(rf_grid.best_score_)
print(rf_grid.best_params_)

In [ ]:
grad_params = {
    'n_estimators' : np.linspace(10, 40, 3, dtype='int'),
    'max_depth': np.linspace(3, 10, 3, dtype='int'),    
    'subsample' : np.linspace(0.3, 0.95, 3),
    'learning_rate' : np.linspace(0.01, 0.3, 5),
}
grad_grid = GridSearchCV(GradientBoostingClassifier(), grad_params, verbose=3, scoring='log_loss', n_jobs=4)
grad_grid.fit(x, y)
print(grad_grid.best_score_)
print(grad_grid.best_params_)

Fitting 3 folds for each of 135 candidates, totalling 405 fits


[Parallel(n_jobs=4)]: Done  24 tasks      | elapsed:   57.3s


In [ ]:
bag = BaggingClassifier(max_samples=0.8, max_features=0.7, n_estimators=40)
#sgd = SGDClassifier(loss='log')
grad = GradientBoostingClassifier(n_estimators=20, max_depth=4, verbose=0, subsample=0.9)
rfc = RandomForestClassifier(max_depth=20, n_estimators=300)
votecf = VotingClassifier([('rf1', rfc),
                           ('bag', bag),
                           ('grad', grad),
                           ('sgd', sgd)
                          ], 'soft')
cross_val_score(votecf, x, y, cv = 5, scoring='log_loss', n_jobs=4)

In [ ]:
rfc = RandomForestClassifier(max_depth=10, n_estimators=50)
rfc.fit(x, y)
print('Top features')
for score, feat in sorted(zip(rfc.feature_importances_, x.columns), reverse=True)[:10]:
    print('{:.3f} {}'.format(score,feat))
print()    
print('Botton features')
for score, feat in sorted(zip(rfc.feature_importances_, x.columns), reverse=True)[-20:]:
    print('{:.5f} {}'.format(score,feat))

In [ ]:
#x_norm = normalize(x)
#svc = SVC(probability=True)
#cross_val_score(svc, x, y, cv=5, verbose=3, scoring='log_loss')

In [ ]:
#pca = PCA(0.999)
#x_pca = pca.fit_transform(x)
#x_pca.shape